In [ ]:
import pandas as pd # library for data analysis
from pandas import DataFrame
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude value
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, confusion_matrix,r2_score
from sklearn import preprocessing
from sklearn.datasets import make_regression
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
import seaborn as sns
from matplotlib.ticker import NullFormatter
from sklearn import metrics
import matplotlib.ticker as ticker
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import mpl_toolkits


print('Libraries imported.')

2018 Tax data to evaluate location 

In [ ]:
#https://data.ftb.ca.gov/California-Personal-Income-Tax/Personal-Income-Tax-Statistics-By-Zip-Code/mriu-wsxf
# 2018 is max available
url='https://data.ftb.ca.gov/resource/mriu-wsxf.json?$where=county="Los Angeles" and taxable_year = 2018 limit 1000000'

In [ ]:
resultsi = requests.get(url).json()
income = pd.DataFrame.from_records(data=resultsi)
# filter columns df.sort(['A', 'B'], ascending=[1, 0])df.sort_values(by='col1', ascending=False)df.sort_values(['Total Due'], inplace=True)
income.sort_values(['zip_code'],ascending=True,inplace=True)
print(income[income.isna().any(axis=1)])

In [ ]:
#Correct Data type

income['zip_code']=income['zip_code'].astype(str)
income['ca_agi'] = income['ca_agi'].astype(int)
income['returns'] = income['returns'].astype(int)
income['total_tax_liability']=income['total_tax_liability'].astype(int)

In [ ]:
#Average Home Price by zip city of LA
#wikiurl="http://www.laalmanac.com/economy/ec37b.php"
wikiurl ="http://www.laalmanac.com/economy/ec37c.php" 
table=""
response2=requests.get(wikiurl)
print(response2.status_code)


In [ ]:
soup = BeautifulSoup(response2.text, 'html.parser')
indiatable=soup.find('table')
df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])

In [ ]:
# Clean data
df.columns = ["zip_code","City/community","2020","2019","2018","2017","2016","2015"]
#df.set_index("zip code",inplace= True)
print(df[df.isna().any(axis=1)])
df.head()

In [ ]:
column=['year','zip_code','city/community','condo price']
cond=[]
cond=pd.DataFrame(columns=column)
ran=df.shape[0]
i=0
for i in range(ran) :
    post = df.iloc[i][0]   
    city = df.iloc[i][1]
    price1 = df.iloc[i][2]
    price2 = df.iloc[i][3]
    price3 = df.iloc[i][4]
    price4 = df.iloc[i][5]
    price5 = df.iloc[i][6]
    price6 = df.iloc[i][7]
    i=i+1    

    cond=cond.append({'year':"2020",'zip_code':post,'city/community':city,'condo price':price1},ignore_index=True)
    cond=cond.append({'year':'2019','zip_code':post,'city/community':city,'condo price':price2},ignore_index=True)
    cond=cond.append({'year':'2018','zip_code':post,'city/community':city,'condo price':price3},ignore_index=True)
    cond=cond.append({'year':'2017','zip_code':post,'city/community':city,'condo price':price4},ignore_index=True)
    cond=cond.append({'year':'2016','zip_code':post,'city/community':city,'condo price':price5},ignore_index=True)
    cond=cond.append({'year':'2015','zip_code':post,'city/community':city,'condo price':price6},ignore_index=True)
cond.dropna(subset=['condo price'],inplace=True)
cond.shape
cond['zip_code']=cond['zip_code'].astype(str)
cond['condo price'] = cond['condo price'].replace({'\$': '', ',': ''}, regex=True).astype(int)    
cond.head()

In [ ]:
# marge income and condo price table˘ note only have one income year˘˘˘˘˘

la_df = pd.merge(cond, income, on='zip_code',how='left')
la_df.drop(['taxable_year','state','county','countylatitude','countylongitude','geo_county','geo_city'],axis = 'columns', inplace=True)
#la_df['price'] = la_df['price'].replace({'\$': '', ',': ''}, regex=True).astype(int)
la_df['income'] = la_df['ca_agi']/la_df['returns']
la_df['liability']=la_df['total_tax_liability']/la_df['returns']
#merge_df['total_tax_liabillity'] = merge_df['total_tax_liabillity'].astype(float)
la_df.drop(['ca_agi','total_tax_liability'],axis = 'columns', inplace=True)
la_df.rename({'zip_code':'zip code'},axis ='columns',inplace =True)


In [ ]:
print(la_df[la_df.isna().any(axis=1)])

In [ ]:
#https://fred.stlouisfed.org/series/CASTHPI
#Unenployment LA County 1990 ~
unemp = pd.read_excel('CALOSA7URN.xls') 
print (unemp.head())


In [ ]:
urlf2020='https://data.lacity.org/resource/kde4-k7yx.json?$limit=100000&$offset=0'
urlf2015='https://data.lacity.org/resource/f37w-ye7d.json?$limit=100000&$offset=0'
urlf2016='https://data.lacity.org/resource/4sbs-dcfn.json?$limit=100000&$offset=0'
urlf2017='https://data.lacity.org/resource/r53k-qp8f.json?$limit=100000&$offset=0'
urlf2018='https://data.lacity.org/resource/e6cg-sqdy.json?$limit=100000&$offset=0'
urlf2019='https://data.lacity.org/resource/rsxb-x48z.json?$limit=100000&$offset=0'

In [ ]:
def getfore(url,wyear):
    resultsf = requests.get(url).json()
    foreclose = pd.DataFrame.from_records(data=resultsf)
    
    
    if wyear == '2015':
        foreclose['regdate'] = foreclose['registereddate'].astype('datetime64[ns]')
        foreclose['yearmonth']=foreclose['regdate'].dt.strftime('%Y-%m')
        foreclose['yearmonth1st'] = foreclose['yearmonth'] + '-01'
        select_col=foreclose[["yearmonth1st","propertyzip","apn"]]
        forecls=select_col.copy()
        forecls.sort_values(['yearmonth1st','propertyzip'],ascending=True,inplace=True)
        forecls.fillna('2015-0w-01',inplace=True)
        
    if wyear == '2016':
        foreclose['regdate'] = foreclose['registereddate'].astype('datetime64[ns]')
        foreclose['yearmonth']=foreclose['regdate'].dt.strftime('%Y-%m')
        foreclose['yearmonth1st'] = foreclose['yearmonth'] + '-01'
        select_col=foreclose[["yearmonth1st","propertyzip","apn"]]
        forecls=select_col.copy()
        forecls.sort_values(['yearmonth1st','propertyzip'],ascending=True,inplace=True)        
        forecls.fillna('2016-02-01',inplace=True)
    if wyear == '2017':
        foreclose['regdate'] = foreclose['registered_date'].astype('datetime64[ns]')
        foreclose['yearmonth']=foreclose['regdate'].dt.strftime('%Y-%m')
        foreclose['yearmonth1st'] = foreclose['yearmonth'] + '-01'
        select_col=foreclose[["yearmonth1st","propertyzip","apn"]]
        forecls=select_col.copy()
        forecls.sort_values(['yearmonth1st','propertyzip'],ascending=True,inplace=True)        
        forecls.fillna('2017-02-01',inplace=True)
    if wyear == '2018':
        foreclose['regdate'] = foreclose['registereddate'].astype('datetime64[ns]')
        foreclose['yearmonth']=foreclose['regdate'].dt.strftime('%Y-%m')
        foreclose['yearmonth1st'] = foreclose['yearmonth'] + '-01'
        select_col=foreclose[["yearmonth1st","propertyzip","apn"]]
        forecls=select_col.copy()
        forecls.sort_values(['yearmonth1st','propertyzip'],ascending=True,inplace=True)
        forecls.fillna('2018-02-01',inplace=True)
    if wyear == '2019':
        foreclose['regdate'] = foreclose['registered_date'].astype('datetime64[ns]')
        foreclose['yearmonth']=foreclose['regdate'].dt.strftime('%Y-%m')
        foreclose['yearmonth1st'] = foreclose['yearmonth'] + '-01'
        select_col=foreclose[["yearmonth1st","propertyzip","apn"]]
        forecls=select_col.copy()
        forecls.sort_values(['yearmonth1st','propertyzip'],ascending=True,inplace=True)
 
        forecls.fillna('2019-02-01',inplace=True)
    if wyear == '2020':
        foreclose['regdate'] = foreclose['registered_date'].astype('datetime64[ns]')
        foreclose['yearmonth']=foreclose['regdate'].dt.strftime('%Y-%m')
        foreclose['yearmonth1st'] = foreclose['yearmonth'] + '-01'
        select_col=foreclose[["yearmonth1st","propertyzip","apn"]]
        forecls=select_col.copy()
        forecls.sort_values(['yearmonth1st','propertyzip'],ascending=True,inplace=True)
        forecls.fillna('2020-02-01',inplace=True)
    return (forecls)

In [ ]:
foreclosea=[]
forecls=[]
forecls=getfore(urlf2015,'2015')

foreclosea = forecls.copy()
forecls=getfore(urlf2016,'2016')
foreclosea = foreclosea.append(forecls)

forecls=getfore(urlf2017,'2017')
foreclosea = foreclosea.append(forecls)

forecls=getfore(urlf2018,'2018')
foreclosea = foreclosea.append(forecls)

forecls=getfore(urlf2019,'2019')
foreclosea = foreclosea.append(forecls)

forecls=getfore(urlf2020,'2020')
foreclosea = foreclosea.append(forecls)

#delete na



In [ ]:
foreclosea.dropna(subset=['yearmonth1st'],inplace=True)

In [ ]:
foreclosea.rename({'propertyzip':'zip code','apn':'foreclosure'},axis='columns',inplace=True)
foreclosea.shape

In [ ]:
foreclose_grp=foreclosea.groupby(['yearmonth1st','zip code']).count().reset_index()

In [ ]:
foreclose_grp['yearmonth1st']=foreclose_grp['yearmonth1st'].astype(str)
foreclose_grp['zip code']=foreclose_grp['zip code'].astype(str)
foreclose_grp.dtypes

In [ ]:
#delete zip = 0 zip = 999, zip = 2018-01-01
indexnm=foreclose_grp[foreclose_grp['zip code']=='0'].index
foreclose_grp.drop(indexnm,inplace = True)
indexnm=foreclose_grp[foreclose_grp['zip code']=='99999'].index
foreclose_grp.drop(indexnm,inplace = True)

indexnm=foreclose_grp[foreclose_grp['zip code']=='2018-01-01'].index
foreclose_grp.drop(indexnm,inplace = True)

In [ ]:
foreclose_grp.groupby(['zip code']).sum()

In [ ]:
#Create unemployment for zip 90016,90018,90034
#Unenployment LA County 1990 ~
#unemp = pd.read_excel('CALOSA7URN.xls') 
#filter 
unemp1=unemp.loc[unemp['observation_date'] > '2014-12-31']

unemp2020 = unemp1.loc[unemp1['observation_date']==unemp1['observation_date'].max()]
rate1 = unemp2020.iloc[0][1]
print(rate1)


In [ ]:
#unemployment by zip

La_Unemployment = 'LAUnemployment.xls'
une = pd.read_excel(La_Unemployment)
une['zip code']=une['zip_code'].astype(str)
une['unemployment']=une['unemployment'].replace({'\%':'',',':''},regex=True).astype(float)
une.drop(labels=['lat','city','rank','zip_code','no','labor','lon'],axis=1,inplace=True)
une['per']=une['unemployment']/rate1


In [ ]:
column=['zip code','unemployment_rate','yearmonth1st']
unemployment=[]
unemployment=pd.DataFrame(columns=column)

In [ ]:
#marge year date and zip unemployment and create new data frame
rang=une.shape[0]
rang1=unemp1.shape[0]
i=0
j = 0
for j in range(rang1):

    for i in range(rang) :
        post = une.iloc[i][1]   
        rate =  une.iloc[i][2] * unemp1.iloc[j][1]
        i=i+1    
        yearmonth = unemp1.iloc[j][0]
        unemployment=unemployment.append({'zip code':post,'unemployment_rate':rate,'yearmonth1st':yearmonth},ignore_index=True)
    j=j +1

unemployment.head()


In [ ]:
unemployment['yearmonth1st']=unemployment['yearmonth1st'].astype(str)
unemployment['zip code']=unemployment['zip code'].astype(str)

In [ ]:
#marge foreclosure and unemployment
mydata = pd.merge(unemployment, foreclose_grp , on=['zip code' , 'yearmonth1st'],how='left')
mydata['foreclosure'].fillna(0,inplace = True)
mydata['year']=mydata['yearmonth1st'].str.slice(start=0,stop=4)
mydata=mydata.loc[mydata['yearmonth1st'] < '2020-01-01' ]
mydata.shape

*************************

In [ ]:
mydata1=pd.merge(mydata,la_df, on =['zip code','year'],how = 'left')
mydata1

In [ ]:
mydata1=pd.merge(mydata,la_df, on =['zip code','year'],how = 'left')
column=['zip code','unemployment_rate','yearmonth','foreclosure','income','condprice','lat','lon']
mydatas=[]
mydatas=pd.DataFrame(columns=column)
#print(mydata1[mydata1.isna().any(axis=1)])
# remove nan
mydata1=mydata1.dropna()
rang=mydata1.shape[0]

i = 0
for i in range(rang):
    post = mydata1.iloc[i][0]   
    rate=  mydata1.iloc[i][1]
    yddate = mydata1.iloc[i][2]
    foreclosure = mydata1.iloc[i][3]
    loc=mydata1.iloc[i][9]
    geo=pd.DataFrame.from_records(loc)
    lon = (geo.coordinates[0])
    lat= (geo.coordinates[1])
    cond = mydata1.iloc[i][6]
    income=mydata1.iloc[i][10]
    i=i+1    
    mydatas=mydatas.append({'zip code':post,'unemployment_rate':rate,'yearmonth':yddate,'foreclosure':foreclosure,'lon':lon,'lat':lat,'condprice':cond,'income':income},ignore_index=True)



rowsx = mydatas.groupby('zip code').agg({'foreclosure':'sum'}).reset_index()

rows1=rowsx.loc[rowsx['foreclosure'] <= 0.0 ]
print (rows1)

In [ ]:
mydatas.describe()

In [ ]:
mydatas.shape

In [ ]:
la_grouped=mydatas.groupby('zip code').agg({'unemployment_rate':'mean','foreclosure':'sum','lat':'mean','lon':'mean'}).reset_index()

la_grouped.head()

In [ ]:
latitude= la_grouped.iloc[0][3]
longitude = la_grouped.iloc[0][4]

In [ ]:
mydatas['zip code']=mydatas['zip code'].astype(int)

In [ ]:
#Show Foreclosure and unemployment on the map
from folium.plugins import MarkerCluster
#geo = pd.DataFrame.from_records(mydata1.loc[3, 'geo_zipcode'])

lamap=folium.Map(location = [latitude,longitude],
default_zoom_start=8)
marker_cluster=MarkerCluster().add_to(lamap)
# add a maker for foreclosur
for lat,lon,poi, forecl in zip(la_grouped['lat'],la_grouped['lon'], la_grouped['zip code'],la_grouped['foreclosure']):
    label =  'foreclosure:' + '{}'.format(forecl) 
#    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [lat, lon],
        popup=label).add_to(marker_cluster)
Zipc='ZIPCodes.geojson'
folium.Choropleth(
    geo_data=Zipc,
    data=la_grouped,
    columns=['zip code','unemployment_rate'],
    key_on='feature.properties.zipcode',
    fill_opacity=0.3,
    line_opacity=0.2,
    line_weight=2,
    legend_name='foreclosure',
    fill_color='RdYlGn'
).add_to(lamap)
folium.LayerControl().add_to(lamap)

lamap



In [ ]:
# Calculate and show pairplot
z=mydatas[['unemployment_rate','condprice','zip code','income','foreclosure']]
sns.pairplot(z, size=2.5)
plt.tight_layout()

In [ ]:
#mydatas.drop(['zip code'],axis = 1,inplace = True)

In [ ]:
#z=mydatas[['unemployment_rate','condprice','income','foreclosure']]
#sns.pairplot(z, size=2.5)
#plt.tight_layout()

In [ ]:
#simple regression zip vs price
msk=np.random.rand(len(mydatas))< 0.8
train=mydatas[msk]
test=mydatas[~msk]
fig = plt.figure(figsize=(10,7))
fig.add_subplot(2,1,1)
plt.scatter(train['yearmonth'],train.unemployment_rate,color='blue')
plt.xlabel("date")
plt.ylabel("unemployment_rate")
plt.show
#simple regression zip vs price
fig = plt.figure(figsize=(10,7))
fig.add_subplot(2,1,1)
plt.scatter(train['yearmonth'],train.foreclosure,color='red')
plt.xlabel("date")
plt.ylabel("foreclosure")
plt.show


#unemployment is very high on bith 20150101 and 20260101lead to overfitting. Therefore I remove those two data
mudatas=mydatas.drop(mydatas[mydatas['yearmonth'] =='2015-01-01'].index,inplace=True)
mudatas=mydatas.drop(mydatas[mydatas['yearmonth'] =='2016-01-01'].index,inplace=True)
mudatas=mydatas.drop(mydatas[mydatas['yearmonth'] =='2017-01-01'].index,inplace=True)
mudatas=mydatas.drop(mydatas[mydatas['yearmonth'] =='2018-01-01'].index,inplace=True)
mudatas=mydatas.drop(mydatas[mydatas['yearmonth'] =='2019-01-01'].index,inplace=True)
#mydatas=mydatas.drop(['yearmonth'],axis =1,inplace=True)
mydatas

In [ ]:

#%matplotlib inline
#bins = np.linspace(mydatas.unemployment_rate.min(),mydatas.unemployment_rate.max(),10)


#g = sns.FacetGrid(mydatas, col='income',hue='foreclosure',palette="Set1",col_wrap = 2)
#g.map(plt.hist,'unemployment_rate',bins=bins,ec ="k")
#g.axes[-1].legend()
#plt.show

In [ ]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="LA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of LA are {}, {}.'.format(latitude, longitude))

Create Model uding multiple regression model

In [ ]:
CLIENT_ID = 'ZH5B0D1UA2ALS2BPY4YFHLDAJTL02J10FTS2STPNUDO5OINP' # your Foursquare ID
CLIENT_SECRET = 'EEKM3BVHOQQ2VGK1GYNNFOOYFRYIN0MCXDGOLE0UAC5CEPFE' # your Foursquare Secret
VERSION = '20201201' # Foursquare API version
LIMIT = 50 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
venues_list=[]    
venues2_list=[]    
venues2_list=[]    

In [ ]:
rows = mydatas.groupby(['zip code','lat','lon']).count().reset_index()

rows2 = rows
rows2.head()

In [ ]:
#4d4b7105d754a06377d81259 out door 4bf58dd8d48988d12f941735 library 4bf58dd8d48988d1ca941735 pizza
#,4bf58dd8d48988d1e0931735,4bf58dd8d48988d1dc931735,4d4b7105d754a06377d81259,4bf58dd8d48988d12f941735,4bf58dd8d48988d1ca941735'

#def getNearbyVenues(names, latitudes, longitudes, radius=500):

Cat ='4bf58dd8d48988d1ca941735,4bf58dd8d48988d16d941735,4bf58dd8d48988d1e0931735,4bf58dd8d48988d1dc931735'
name=la_df['zip code']
radius = 500
for name, loc in zip(la_df['zip code'], la_df['geo_zipcode'] ):
    
        #print(name)
    geo = pd.DataFrame.from_records(data=loc)
    lng = (geo.coordinates[0])
    lat= (geo.coordinates[1])        
        # create the API request URL

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT,
        Cat)
    url
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
                'Neighborhood Latitude', 
                'Neighborhood Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category']
    #return(nearby_venues)

In [ ]:
#4bf58dd8d48988d16d941735 cafa  4bf58dd8d48988d1e0931735 coffee shop 4bf58dd8d48988d1dc931735 tea room
#4d4b7105d754a06377d81259 out door 4bf58dd8d48988d12f941735 library 4bf58dd8d48988d1ca941735 pizza
                            
venues1_list=[]    
Cat ='4d4b7105d754a06377d81259,4bf58dd8d48988d1ca941735'
name=rows2
radius = 500
for name, lat,lng in zip(rows2['zip code'], rows2['lat'],rows2['lon'] ):
    
        #print(name)
  #  geo = pd.DataFrame.from_records(data=loc)
  #  lng = (geo.coordinates[0])
  #  lat= (geo.coordinates[1])        
        # create the API request URL

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT,
        Cat)
    url
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venues_list in venues_list for item in venues_list])
nearby_venues.columns = ['Neighborhood', 
                'Neighborhood Latitude', 
                'Neighborhood Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category']

In [ ]:
nearby_venues.to_csv('venuesdatas.csv')

In [ ]:
nearby_venues.groupby(['Neighborhood','Venue Category']).count()

In [ ]:
# do not run foresquare because of limit, use backup
nearby_venues=pd.read_csv('venuesdata.csv')
nearby_venues['Neighborhood']=nearby_venues['Neighborhood'].astype(str)


In [ ]:
# one hot encoding
onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
onehot1 = nearby_venues['Neighborhood'] 
outdoor = onehot['Waterfront']+onehot['Waterfall']+onehot['Bay'] + onehot['Beach']+onehot['Nature Preserve']+onehot['Outdoors & Recreation']+onehot['Park']+onehot['Trail']+onehot['Bike Trail']
Sports = onehot['Recreation Center']+onehot['Track']+onehot['Sports Club']+onehot['Track']+onehot['Yoga Studio']+onehot['Climbing Gym']+onehot['Cycle Studio']+onehot['Gym / Fitness Center']+onehot['Pilates Studio']
Coffe = onehot['Café']+onehot['Coffee Shop']+onehot['Tea Room']
#Lib = onehot['College Library']+onehot['Library']
#onehot.drop(labels=['Neighborhood'],axis=1,inplace=True)
#onehot2=onehot2.append({'zip_code':onehot1,'beach':Beach,'sports':Sports,'coffee':Coffe,'librart':Lib},ignore_index=True)
onehot.insert(0,'Neighborhood',onehot1)
onehot.insert(1,'outdoor',outdoor)
onehot.insert(2,'sports',Sports)
onehot.insert(3,'Coffee/tea',Coffe)
#onehot.insert(4,'lib',Lib)
onehot2 = onehot[['Neighborhood','Coffee/tea','outdoor','sports']]
onehot2.head




In [ ]:
la_grouped = onehot2.groupby('Neighborhood').mean().reset_index()
la_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = la_grouped['Neighborhood']

for ind in np.arange(la_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
kclusters = 5

la_grouped_clustering = la_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
neighborhoods_venues_sorted.dtypes

In [ ]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.rename({'Neighborhood':'zip code'},axis ='columns',inplace =True)
la_merged = la_df

## merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
la_merged = la_merged.join(neighborhoods_venues_sorted.set_index('zip code'), on='zip code')

#neighborhoods_venues_sorted.head() # check the last columns!
la_merged.head()

In [ ]:

la_merged['Cluster Labels'].fillna(0, inplace=True)
la_merged['Cluster Labels']=la_merged['Cluster Labels'].astype(int)
la_merged.head()

In [ ]:
la_merged.dtypes

In [ ]:
lamap=folium.Map(location = [latitude,longitude],
default_zoom_start=12)
Zipc='ZIPCodes.geojson'
folium.Choropleth(
    geo_data=Zipc,
    data=la_merged,
    columns=['zip code','condo price'],
    key_on='feature.properties.zipcode',
    fill_opacity=0.3,
    line_opacity=0.2,
    line_weight=2,
    legend_name='condo price',
    fill_color='RdYlGn'
).add_to(lamap)
folium.LayerControl().add_to(lamap)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for loc, poi, cluster ,hincome ,liab,hprice in zip(la_merged['geo_zipcode'], la_merged['zip code'], la_merged['Cluster Labels'],la_merged['income'],la_merged['liability'],la_merged['condo price']):
    geo = pd.DataFrame.from_records(data=loc)
    lon = (geo.coordinates[0])
    lat= (geo.coordinates[1])
    #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster)+' Unemployment ' + unemp +'Income '+income +'price'+price , parse_html=True)
    label =  str(poi) + ' Cluster ' + str(cluster) + 'liability:' + '{}'.format(liab) +'Income :'+'{}'.format(hincome) +'condo price: '+'{}'.format( hprice)
    label = folium.Popup(label, parse_html=True)
    if cluster != 0 and hprice < 600000:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(lamap)
lamap

I am going to use 3 zip code 90744, ,90008,90068 to create Model

In [ ]:
la_merged.loc[la_merged['Cluster Labels'] == 4, la_merged.columns[[0] +list(range(1, la_merged.shape[1]))]]

In [ ]:
la_grp=la_merged[la_merged['Cluster Labels']== 4]
la_grp.groupby('zip code').sum()

In [ ]:
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('foreclosure')
    plt.ylabel('Proportion ')

    plt.show()
    plt.close()

In [ ]:

#Create mulitiple regression with train & test
mydataz=mydatas

lr1 = LinearRegression()
label=mydataz['foreclosure']
train1 = mydataz.drop(['foreclosure','lat','lon','yearmonth','zip code'],axis=1)
    
#Creating train / test set
x_train, x_test, y_train, y_test = train_test_split(train1, label, test_size=0.20, random_state=4)

lr1.fit(x_train, y_train)
print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])
yhat_train = lr1.predict(x_train)
yhat_test = lr1.predict(x_test)

Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_train, yhat_train, "Actual Values (Train)", "Predicted Values (Train)", Title)
Title='Distribution  Plot of  Predicted Value Using Test Data vs Data Distribution of Test Data'
DistributionPlot(y_test,yhat_test,"Actual Values (Test)","Predicted Values (Test)",Title)
    # visualizing residuals
fig = plt.figure(figsize=(10,5))
residuals = (y_test- yhat_test)
sns.distplot(residuals)
#    print (mydataz.head())
#    house1 =train1.loc[(train1['condprice'] > 308300)]
print('The R-square Linear is: ', lr1.score(train1, mydataz['foreclosure']))
#compare actual output values with predicted values
# evaluate the performance of the algorithm (MAE - MSE - RMSE)

print('MAE:', metrics.mean_absolute_error(y_test, yhat_test))  
print('MSE:', metrics.mean_squared_error(y_test, yhat_test))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, yhat_test)))
print('VarScore:',metrics.explained_variance_score(y_test,yhat_test))


In [ ]:

params = {'n_estimators': 500,
          'max_depth': 6,
          'min_samples_split': 2,
          'learning_rate': 0.09,
          'loss': 'ls'}
   # params = {'n_estimators': 500,
  #        'max_depth': 4,
  #        'min_samples_split': 2,
  #        'learning_rate': 0.1,
  #        'loss': 'quantile',
  #        'alpha': .78}
label=mydataz['foreclosure']
train1 = mydataz.drop(['foreclosure','lat','lon','yearmonth','zip code'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(train1, label, test_size=0.30, random_state=6)
clf = ensemble.GradientBoostingRegressor(**params)
clf.fit(x_train,y_train)    
clf.score(x_train,y_train)
yhat_test2=clf.predict(x_test)
mse = mean_squared_error(y_test, clf.predict(x_test))
print('The GRADIENT BOOST R-square is: ', clf.score(x_train,y_train))

print('MAE:', metrics.mean_absolute_error(y_test, yhat_test2))  
print('MSE:', metrics.mean_squared_error(y_test, yhat_test2))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, yhat_test2)))
print('VarScore:',metrics.explained_variance_score(y_test,yhat_test2))

test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(clf.staged_predict(x_test)):
    test_score[i] = clf.loss_(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, clf.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
fig.tight_layout()
plt.show()
    #check


In [ ]:
#NOW I WILL EVALUATE ZIP CODE 90008/90023/90018


In [ ]:
#check result


house1 =train1.loc[(mydatas['zip code']== 90023 ) & ( mydatas['unemployment_rate'] > 0.0) & ( mydatas['unemployment_rate'] < 4.5)]
print('Liner :',lr1.predict(house1).mean())
print('Grad :',clf.predict(house1).mean())
house2 =mydatas.loc[(mydatas['zip code']== 90023 ) & ( mydatas['unemployment_rate'] > 0.0) & ( mydatas['unemployment_rate'] < 4.5)]
#print (house2)
print(house2.mean())


In [ ]:
#check

house1 =train1.loc[(mydatas['zip code']== 90018 ) & ( mydatas['unemployment_rate'] > 5.0) & ( mydatas['unemployment_rate'] < 5.5)]
print('Liner :',lr1.predict(house1).mean())
print(clf.predict(house1).mean())
house2 =mydatas.loc[(mydatas['zip code']== 90018 ) & ( mydatas['unemployment_rate'] > 5.0) & ( mydatas['unemployment_rate'] < 5.5)]
#print (house2)
print(house2.mean())

In [ ]:
house1 =train1.loc[(mydatas['zip code']== 90008 ) & ( mydatas['unemployment_rate'] > 0.0) & ( mydatas['unemployment_rate'] < 4.5)]
print('Liner :',lr1.predict(house1).mean())
print('Grad :',clf.predict(house1).mean())
house2 =mydatas.loc[(mydatas['zip code']== 90008 ) & ( mydatas['unemployment_rate'] > 0.0) & ( mydatas['unemployment_rate'] < 4.5)]
#print (house2)
print(house2.mean())